In [72]:
import random
import numpy as np
import math

In [73]:
class RowVectorFloat:
    def __init__(self, elements):
        self.elements = list(elements)

    def __str__(self):
        return ' '.join(map(str, self.elements))

    def __len__(self):
        return len(self.elements)

    def __getitem__(self, index):
        return self.elements[index]

    def __setitem__(self, index, value):
        self.elements[index] = value

    def __add__(self, other):
        if len(self) != len(other):
            raise Exception("Vector dimensions must match for addition")
        result_elements = [x + y for x, y in zip(self.elements, other.elements)]
        return RowVectorFloat(result_elements)

    def __sub__(self, other):
        if len(self) != len(other):
            raise Exception("Vector dimensions must match for subtraction")
        result_elements = [x - y for x, y in zip(self.elements, other.elements)]
        return RowVectorFloat(result_elements)

    def __mul__(self, scalar):
        result_elements = [x * scalar for x in self.elements]
        return RowVectorFloat(result_elements)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

In [74]:
class SquareMatrixFloat:
    def __init__(self, n):
        self.matrix = [RowVectorFloat([0] * n) for _ in range(n)]
        self.size = n

    def __str__(self):
        s = "The matrix is: \n"
        s = s + '\n'.join(['\t'.join([f"{elem:.2f}" if elem != 0 else '0' for elem in row.elements]) for row in self.matrix])
        return s
        # return '\n'.join(map(str, self.matrix))
    
    def __setitem__(self, index, value):
        self.matrix[index] = value

    def sampleSymmetric(self):
        for i in range(self.size):
            for j in range(i,self.size):
                self.matrix[i][j] = self.matrix[j][i] = random.random() if i != j else random.uniform(1, self.size)
    
    def toRowEchelonForm(self):
        pivot = 0
        for i in range(self.size):
            # Find the pivot row
            pivot_row = None
            for j in range(i, self.size):
                if self.matrix[j][pivot] != 0:
                    pivot_row = j
                    break
            
            # Swap rows if necessary
            if pivot_row is not None:
                self.matrix[i], self.matrix[pivot_row] = self.matrix[pivot_row], self.matrix[i]
            else:
                continue
            
            # Scale the pivot row
            pivot_element = self.matrix[i][pivot]
            self.matrix[i] = self.matrix[i] * (1 / pivot_element)
            
            # Eliminate below the pivot
            for j in range(i + 1, self.size):
                factor = self.matrix[j][pivot]
                self.matrix[j] = self.matrix[j] - (self.matrix[i] * factor)
            
            pivot += 1
        
    def isDRDominant(self):
        for i in range(len(self.matrix)):
            diagonal_value = abs(self.matrix[i].elements[i])
            row_sum = sum(abs(x) for x in self.matrix[i].elements if x != diagonal_value)
            if row_sum > diagonal_value:
                return False
        return True

    def jSolve(self, b, m):
        if not self.isDRDominant():
            raise Exception("Not solving because convergence is not guranteed.")
            
        x = [0.0] * self.size
        x_kplus1 = [0.0] * self.size
        err = []

        #   Iteratively compute xi(k+1) using formula (perform m iterations)
        for _ in range(m):
            for i in range(self.size):
                sum = 0
                for j in range(self.size):
                    if i != j:
                        sum += self.matrix[i][j]*x[j]
                x_kplus1[i] = (b[i]-sum)/self.matrix[i][i]

            x=x_kplus1

            #   Find value of term ∥Ax(k) − b∥2
            e = 0
            for i in range(self.size):
                s = 0
                for j in range(self.size):
                    s+=self.matrix[i][j]*x[j]
                e += (b[i] - s) ** 2
            err.append(math.sqrt(e))

        return err,x
    
    def gsSolve(self, b, m):
    #   Here, new (k+1) components already available are used
        x = [0.0] * self.size
        err = []

        for _ in range(m):
            x_kplus1 = [0.0] * self.size
            for i in range(self.size):
                s = 0
                s2 = 0
                
                for j in range(i):
                        s += self.matrix[i][j] * x_kplus1[j]

                for j in range(i+1,self.size):
                        s2 += self.matrix[i][j]*x[j]

                x_kplus1[i]=(b[i]-s-s2)/self.matrix[i][i]

            x = x_kplus1
            e = 0
            for i in range(self.size):
                sum = 0
                for j in range(self.size):
                    sum += self.matrix[i][j]*x[j]

                e += (b[i]-sum) ** 2

            err.append(math.sqrt(e))
            
        return err,x


In [81]:
s = SquareMatrixFloat(4)
s.sampleSymmetric()
print(s.isDRDominant())
print(s)
(err, x) = s.gsSolve([1, 2, 3, 4], 10)
print(x)
print(err)

False
The matrix is: 
2.47	0.73	0.48	0.92
0.73	1.03	0.09	0.48
0.48	0.09	1.13	0.99
0.92	0.48	0.99	1.33
[-1.206013212686822, 1.4258403752533182, 0.5972529363351103, 2.881371904155708]
[2.7343401335475543, 1.0164353773472417, 0.5893311006066657, 0.4349625781236505, 0.3358409146046756, 0.26017381017643776, 0.2013454392630306, 0.15569928823341928, 0.1203584354212216, 0.09302546555679948]
